## Load packages 



In [ ]:
knitr::opts_chunk$set(echo = TRUE)

# for data reading/manipulation 
library(dplyr)
library(tidyr)
library(readr)
library(tibble)
library(janitor)
# for spatial data and gis
library(sf)
library(ggmap)
library(ggplot2)
library(ggspatial)
library(ggspatial)
library(spdep)
library(leaflet) 
library(RColorBrewer)
library(tmap)


## Explore the data using ggmap and ggplot (Google API)

### Setting up a an API 


Setting up a connetion to Google's API;


1. First: set up an account with Google.


2. Second: to obtain an API go to [https://cloud.google.com/maps-platform/] and follow the onscreen instructions. This documentation shows you how to input the requisite information (e.g. your API key) into R, and it also shows you a few tools that can help you work with the credentialing. In summary the onscreen instructions are as follows. 
 - click 'get started' 
 - click the credentials tab on the left hand side of the screen 
 - click 'create credentials at the top of the screen (visible by the '+' symbol)
 - Select 'API key'
 - Select copy and now you have your personal API key 
 

3. To authorise the API you have to enable some of Google's API which are required for the project. This includes 
- Google Maps JavaScript API
- Google Maps Geocoding API
- Google Maps Geolocation API
- Google Maps Places API

If you do not authorise the above API you will recieve and error message as such "Geocoding Service: This API project is not authorised to use the API". To authorise said APIs go to 
a. https://console.cloud.google.com/apis/dashboard?project=missing-314119
b. Click '+ Enable APIS and SERVICES' at the top of the screen
c. Search for each API listed above and click the 'Enable button' 

Once all APIs are enabled, wait 5 minutes for changes to take place, and then move on to the next step to register your Google API to ggmaps. 


4. Third; you need to tell ggmap about your API by using the 'register_google()' function. It would look something like *register_google(key = "hahfuibfiu324898249dbhsgag")* (this is a fake key). This will then set your API key for the current session, Every time you restart R you will have to request a new API key or you can set it permanently by adding 'write = TRUE', the full code being....

*register_google(key = "[your key]", write = TRUE)*

For more information on how to register your decive to the Google API please visit [https://rdrr.io/cran/ggmap/man/register_google.html]. It is incredibly important to not that each API is private and should not be shared with anyone. Users should also be aware that ggmap has no mechanism with which to safeguard the private key once registered with R. 

----------------------------------------------------------------

The basic idea driving ggmap is to take a downloaded map image, plot it as a context layer using ggplot2, and then plot additional content layers of data, statistics, or models on top of the map. In ggmap, downloading a map as an image and formatting the image for plotting is done with the get_map function. More specifically, the get_map is a wrapper function

It is important to note that when using ggmap, users have to first setup an account with Google, enable the relevant API, and then tell R about the user's setup. Do not worry about doing this for this workshop as we will demonstrate the code below. If you would like to run this in your own time then please refer to the steps in 'Google_API.rmd' which details how to obtain your API and how to enable the services. 


In [ ]:
# ## Ariel Map of Surrey
qmplot(longitude, latitude, data = crime, colour = crime_type, size = I(3), darken = .3)
# 

# ## Lets just say you were interested in a specific area (in this example we will use Crawley 002B)
# 
# ## Ariel Map of Crawley 002B
 geocode("Crawley 002B")
# 

Crawley <- c(long = -0.152210, lat = 51.15813)
map <- get_map(Crawley, zoom = 13, scale = 1)
ggmap(map)
# 
ggmap(map) +
  geom_point(aes(longitude, latitude), data = crime) 
# 
# 
# ## Colour the Crime Type
# 

ggmap(map) +
   geom_point(aes(longitude, latitude, colour = crime_type), data = crime) 

# 

 ggmap(map) +
   geom_point(aes(longitude, latitude, size = crime_type, colour = crime_type), data = crime) 


### Binning data

Binning, can be thought of as a two-dimensional histogram (shading of the bins take the heights of the bars). You first need to convert the sf data.frame geometry column into a data.frame with separate x, y columns. 

How do you separate the coordinates? 

Luckily a function to do this already exists [https://github.com/r-spatial/sf/issues/231]. The below code is converting a sfc_point to seperate x, y columns 


In [ ]:
sfc_as_cols <- function(x, names = c("x","y")) {
  stopifnot(inherits(x,"sf") && inherits(sf::st_geometry(x),"sfc_POINT"))
  ret <- sf::st_coordinates(x)
  ret <- tibble::as_tibble(ret)
  stopifnot(length(names) == raster::ncol(ret))
  x <- x[ , !names(x) %in% names]
  ret <- setNames(ret,names)
  dplyr::bind_cols(x,ret)
}

sf_seperate <- sfc_as_cols(sf, c("lng", "lat")) 


ggplot(sf_seperate, aes(lng, lat)) +   
  annotation_map_tile() +
  stat_binhex(bins = 30) +                                           
  scale_fill_gradientn(colours = c("white","red"), name = "Frequency")   


#hexagonal = stat_binhex() 
#rectangle = stat_bin2d()
#heat = stat_density2d()  


### Interactive Maps; Leaflet

Leaflet is one of the most popular open-source JavaScript libraries for interactive maps. For more information you can view this link here [https://rstudio.github.io/leaflet/]


In [ ]:
## Subsetting for just ASB 
asb <- subset(crime, crime_type == "Anti-social behaviour")

m <- leaflet(data = asb) %>%
  addProviderTiles("Stamen.Toner") %>% 
  addMarkers(lng=~longitude, lat=~latitude, popup=~as.character(location), label = ~as.character(location))
m


### Other imporant functions 

- Jittering: 

Jittering indeed means just adding random noise to a vector of numeric values, by default this is done in jitter-function by drawing samples from the uniform distribution. The range of values in the jittering is chosen according to the data, if amount-parameter is not provided. It helps to grasp where the density of observations is high. 

There are a few packages that offer this method including the 'geom_jitter' function found in the ggplot2 package [https://ggplot2.tidyverse.org/reference/geom_jitter.htm]. Additionally, the 'rjitter' function under the spatsat package. The function rjitter is generic, with methods for point patterns (described here) and for some other types of geometrical objects. Each of the points in the point pattern X is subjected to an independent random displacement. More information can be found here; [https://rdrr.io/cran/spatstat.geom/man/rjitter.html]

-  st_intersect():

This function is also under the sf package and is used to intersect two objects between two sets of objects. More information can be found here [https://r-spatial.github.io/sf/reference/geos_binary_ops.htm]
